In [182]:
import PyPDF2
import re
import pandas as pd


def parse_timetable(file_path, output_path):
    # read your file
    pdf = PyPDF2.PdfReader(open(file_path, 'rb'))

    replace = ["Syllabus/Component Code Duration Date Session",
            "Sessions:  AM morning        PM afternoon        EV evening\nIG Cambridge IGCSE    OL Cambridge O Level    \nAS Cambridge International AS Level    AL Cambridge International A Level    \nCambridge Final Exam Timetable June 2023", "Syllabus view (A–Z)"]

    data = []

    for i in range(9, len(pdf.pages)):
        text = pdf.pages[i].extract_text()
        for replace_line in replace: text = text.replace(replace_line, '')
        temp = list(filter(None, text[:-3].splitlines()))
        
        text = []
        
        def dontJoin(x): return x.split()[0] == 'Cambridge' or len(x) == 1 or x[-2:] in ['AM', 'PM', 'EV']
        
        flag = False
        for i in range(len(temp)):
            if dontJoin(temp[i]):
                if flag or len(temp[i]) == 1 or temp[i][-9:] == 'Continued': flag = False;continue
                text.append(temp[i])
            else: text.append(temp[i] + " " + temp[i+1]);flag = True;i+=1
        
        data.extend(text)

    data_string = "\n".join(data)

    titles = re.findall(r'Cambridge[a-zA-Z ]*\n', data_string)

    # split the data_string based on the titles
    data = re.split(r'Cambridge[a-zA-Z ]*\n', data_string)[1:]

    data = [[[titles[i].strip(), x, *re.findall(
            r'([a-zA-Z0-9 ()–&]*)[ ]?([0-9]{4}\/[0-9]{2}) ([0-9]*?h?[ ]?[0-9]*?m?) (.*)', x)] for x in sorted(d.splitlines())]
            for i, d in enumerate(data)]

    data_map = {title.strip(): d for title, d in zip(titles, data)}

    # flatten data
    def flatten(l): return [item for sublist in l for item in sublist]

    flat_data = flatten(data)

    flat_data = [[i[0], *i[2]] for i in flat_data]

    # convert the data_map to a dataframe
    df = pd.DataFrame(flat_data)
    df.columns = ['type', 'subject', 'code', 'duration', 'date']
    df.to_csv(output_path, index=False)
    return df

In [183]:
df = parse_timetable(file_path='file.pdf', output_path='timetable.csv')
df

,type,subject,code,duration,date
0,Cambridge IGCSE,Accounting (Multiple Choice),0452/12,1h 15m,Friday 26 May 2023 AM
1,Cambridge IGCSE,Accounting,0452/22,1h 45m,Friday 19 May 2023 AM
2,Cambridge IGCSE,Additional Mathematics,0606/12,2h,Tuesday 23 May 2023 AM
3,Cambridge IGCSE,Additional Mathematics,0606/22,2h,Tuesday 30 May 2023 AM
4,Cambridge IGCSE,Arabic (Listening – Multiple Choice),0544/12,50m,Thursday 18 May 2023 PM
...,...,...,...,...,...
368,Cambridge International A Level,Travel & Tourism,9395/32,1h 30m,Wednesday 24 May 2023 PM
369,Cambridge International A Level,Travel & Tourism,9395/42,1h 30m,Friday 26 May 2023 PM
370,Cambridge International A Level,Urdu,9686/02,1h 45m,Tuesday 25 April 2023 PM
371,Cambridge International A Level,Urdu,9686/03,1h 30m,Wednesday 17 May 2023 PM
